In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math
import seaborn as sns
from sklearn import linear_model
from sklearn import preprocessing
from matplotlib import rcParams

%matplotlib inline
sns.set_style('darkgrid')
rcParams['figure.figsize'] = 10, 7

In [2]:
# load the data again. Keep air quality data, drop the index column and any missing data columns

df = pd.read_csv('https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/credit-card-defaults.csv').iloc[:,1:].dropna()

In [3]:
df.head()

,default,student,balance,income
0,No,No,729.526495,44361.625074
1,No,Yes,817.180407,12106.134700
2,No,No,1073.549164,31767.138947
3,No,No,529.250605,35704.493935
4,No,No,785.655883,38463.495879


In [4]:
# Recode strings to numeric.
df['default'] = np.where(df['default'] == 'Yes', 1, 0)
df['student'] = np.where(df['student'] == 'Yes', 1, 0)
names = df.columns
df = pd.DataFrame(preprocessing.scale(df), columns=names)

c:\users\gebruiker\appdata\local\programs\python\python36-32\lib\site-packages\ipykernel_launcher.py:5: DataConversionWarning: Data with input dtype int32, float64 were all converted to float64 by the scale function.
  """


In [5]:
df.head()

,default,student,balance,income
0,-0.185599,-0.645936,-0.218835,0.813187
1,-0.185599,1.548141,-0.037616,-1.605496
2,-0.185599,-0.645936,0.492410,-0.131212
3,-0.185599,-0.645936,-0.632893,0.164031
4,-0.185599,-0.645936,-0.102791,0.370915


In [6]:
# Define the training and test sizes.
trainsize = int(df.shape[0] / 2)
df_test = df.iloc[trainsize:, :].copy()
df_train = df.iloc[:trainsize, :].copy()

Y_train = df_train['income'].values.reshape(-1, 1)
X_train = df_train.loc[:, ~(df_train.columns).isin(['income'])]

In [7]:
# Make some new features to capture potential quadratic and cubic relationshps
df_train['balance_student'] = df_train['balance'] * df['student']
df_train['balance_default'] = df_train['balance'] * df['default']
df_train['student_default'] = df['student'] * df['default']
df_train['balance_sqrt'] = (df['balance'] + 100) * .5
df_train['balance2'] = (df['balance'] + 100) ** 2
df_train['balance3'] = (df['balance'] + 100) ** 3

In [8]:
X_train2 = df_train.loc[:, ~(df_train.columns).isin(['income'])]

In [9]:
# Test the simpler mdoel with smaller coefficients
Y_test = df_test['income'].values.reshape(-1, 1)
X_test = df_test.loc[:, ~(df_test.columns).isin(['income'])]



In [10]:
df_test['balance_student'] = df_test['balance'] * df_test['student']
df_test['balance_default'] = df_test['balance'] * df_test['default']
df_test['student_default'] = df_test['student'] * df_test['default']
df_test['balance_sqrt'] = (df_test['balance'] + 100) ** .5
df_test['balance2'] = (df_test['balance'] + 100) ** 2
df_test['balance3'] = (df_test['balance'] + 100) ** 3
X_test2 = df_test.loc[:, ~(df_test.columns).isin(['income'])]

In [11]:
# Small number of parameters.
lass = linear_model.Lasso(alpha=.35)
lassfit = lass.fit(X_train, Y_train)
print('R² for the model with few features:')
print(lass.score(X_train, Y_train))
origparams = np.append(lassfit.coef_, lassfit.intercept_)
print('\nParameter estimates for the model with few features:')
print(origparams)

# Large number of parameters.
lassBig = linear_model.Lasso(alpha=.35)
lassBig.fit(X_train2, Y_train)
print('\nR² for the model with many features:')
print(lassBig.score(X_train2, Y_train))
origparams = np.append(lassBig.coef_, lassBig.intercept_)
print('\nParameter estimates for the model with many features:')
print(origparams)

R² for the model with few features:
0.4500625793011847

Parameter estimates for the model with few features:
[-0.         -0.40657726 -0.          0.00114596]

R² for the model with many features:
0.44363376712897057

Parameter estimates for the model with many features:
[ 0.00000000e+00 -3.89351238e-01  0.00000000e+00 -0.00000000e+00
  0.00000000e+00 -0.00000000e+00  0.00000000e+00 -2.77688887e-04
 -7.09158792e-07  3.48711577e+00]


In [12]:
print(lass.score(X_test, Y_test))

print(lassBig.score(X_test2, Y_test))

0.44553225151184195
0.4380466345914473


In [15]:
# store estimates
#estimates = np.append(ridg)

for lambd in range(1, 50, 2):
    ridge = linear_model.Ridge(alpha=lambd, fit_intercept=False)
    lasso = linear_model.Lasso(alpha=lambd/50, fit_intercept=False)
    ridge.fit(X_train2, Y_train)
    lasso.fit(X_train2, Y_train)
    print('Ridge Regression with lambda of {}:'.format(lambd))
    print(ridge.score(X_test2, Y_test))
    print('Lasso Regression with lambda of {}:'.format(lambd/50))
    print(lasso.score(X_test2,Y_test))
    print()

c:\users\gebruiker\appdata\local\programs\python\python36-32\lib\site-packages\sklearn\linear_model\coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


Ridge Regression with lambda of 1:
-2.8309454262006724
Lasso Regression with lambda of 0.02:
0.5632640522994256



c:\users\gebruiker\appdata\local\programs\python\python36-32\lib\site-packages\sklearn\linear_model\coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


Ridge Regression with lambda of 3:
-0.6947304657561983
Lasso Regression with lambda of 0.06:
0.5607762796171338



c:\users\gebruiker\appdata\local\programs\python\python36-32\lib\site-packages\sklearn\linear_model\coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


...

c:\users\gebruiker\appdata\local\programs\python\python36-32\lib\site-packages\sklearn\linear_model\coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


Ridge Regression with lambda of 45:
0.5500181215068853
Lasso Regression with lambda of 0.9:
0.003936996296315387



c:\users\gebruiker\appdata\local\programs\python\python36-32\lib\site-packages\sklearn\linear_model\coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


Ridge Regression with lambda of 47:
0.5511435765126644
Lasso Regression with lambda of 0.94:
0.003927347083147348

Ridge Regression with lambda of 49:
0.55213664366349
Lasso Regression with lambda of 0.98:
0.003917694912374925



c:\users\gebruiker\appdata\local\programs\python\python36-32\lib\site-packages\sklearn\linear_model\coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
